In [62]:
import numpy as np
from openfermion.chem import geometry_from_pubchem
from quri_parts.chem import cas
from quri_parts.pyscf.mol import get_spin_mo_integrals_from_mole
from quri_parts.openfermion.mol import get_qubit_mapped_hamiltonian
from quri_parts.openfermion.ansatz import TrotterUCCSD
from quri_parts.core.operator import get_sparse_matrix
from pyscf import gto, scf, mcscf, cc, ci
from quri_parts_qsci import qsci
from quri_parts.chem.mol import ActiveSpaceMolecularOrbitals, ActiveSpace
from quri_parts.pyscf.mol import PySCFMolecularOrbitals
from quri_parts.core.state import quantum_state

from typing import Sequence

import numpy.typing as npt

1. Hamiltonian generation. (Refer to [tutorial](https://quri-sdk.qunasys.com/docs/tutorials/quri-parts/quantum-chemistry/hamiltonian))
2. Use PySCF to compute CASCI energy (the same as computing exact excitation energy)
3. Use PySCF to compute CCSD amplitudes in the case of non-trivial active space. 
4. Describe QSCI algorithm for excited state (assume start from HF)
5. Describe cost function for QSCI (remember to stress that grad optimizer doesn't make sense)
6. Run the algorithm & plot.

# Generate Hamiltonian of AB (azobenzene)
1. Hamiltonian
2. Exact ground state energy
3. Approximated energy includes electron correlation effects from single and double excitations. 

In [59]:
from scipy.sparse.linalg import eigsh

atom = geometry_from_pubchem("azobenzene") 
mole = gto.M(atom=atom)
mf = scf.RHF(mole).run()

active_space = cas(4, 4)
hamiltonian, mapping = get_qubit_mapped_hamiltonian(
    *get_spin_mo_integrals_from_mole(mole, mf.mo_coeff, active_space)
)
print("Hamiltonian: ", hamiltonian)

converged SCF energy = -562.053344724964
Hamiltonian:  (-561.216361546909+0j)*I + (0.07929207448090994+0j)*Z0 + (0.07929207448090994+0j)*Z1 + (0.04173779219560608+0j)*Z2 + (0.04173779219560608+0j)*Z3 + (-0.11119726823032844+0j)*Z4 + (-0.1111972682303285+0j)*Z5 + (-0.08721715886294745+0j)*Z6 + (-0.08721715886294745+0j)*Z7 + (0.054889797130520154+0j)*Z0 Z1 + -0.0015964182709082898*X0 Z1 X2 + -0.0015964182709082898*Y0 Z1 Y2 + (0.029990484826442333+0j)*Z0 Z2 + (0.05354451192306399+0j)*Z0 Z3 + -1.999560816308268e-05*X0 Z1 Z2 Z3 X4 + -1.999560816308268e-05*Y0 Z1 Z2 Z3 Y4 + (0.0499629720365348+0j)*Z0 Z4 + (0.05100356011927254+0j)*Z0 Z5 + 0.014257630971934407*X0 Z1 Z2 Z3 Z4 Z5 X6 + 0.014257630971934407*Y0 Z1 Z2 Z3 Z4 Z5 Y6 + (0.04295665359379545+0j)*Z0 Z6 + (0.05489765632565704+0j)*Z0 Z7 + (0.05354451192306399+0j)*Z1 Z2 + -0.0015964182709082898*X1 Z2 X3 + -0.0015964182709082898*Y1 Z2 Y3 + (0.029990484826442333+0j)*Z1 Z3 + (0.05100356011927254+0j)*Z1 Z4 + -1.999560816308268e-05*X1 Z2 Z3 Z4 X5 +

Compute exact excited state with PySCF.

In [61]:
mc = mcscf.CASCI(mf, active_space.n_active_orb, active_space.n_active_ele)
mc.fcisolver.nstates = 11
mc.kernel(verbose=0)
mc.e_tot

CASCI state   0  E = -562.057671219962  E(CI) = -2.20745364895083  S^2 = 0.0000000
CASCI state   1  E = -561.829824515092  E(CI) = -1.97960694408039  S^2 = 2.0000000
CASCI state   2  E = -561.812652554593  E(CI) = -1.96243498358126  S^2 = 0.0000000
CASCI state   3  E = -561.752682574618  E(CI) = -1.90246500360593  S^2 = 2.0000000
CASCI state   4  E = -561.747970360663  E(CI) = -1.89775278965169  S^2 = 2.0000000
CASCI state   5  E = -561.740607983118  E(CI) = -1.89039041210674  S^2 = 0.0000000
CASCI state   6  E = -561.734241645807  E(CI) = -1.88402407479578  S^2 = 2.0000000
CASCI state   7  E = -561.682073804748  E(CI) = -1.83185623373618  S^2 = 0.0000000
CASCI state   8  E = -561.642062543434  E(CI) = -1.79184497242193  S^2 = 0.0000000
CASCI state   9  E = -561.607468237758  E(CI) = -1.75725066674659  S^2 = 6.0000000
CASCI state  10  E = -561.586873951967  E(CI) = -1.73665638095497  S^2 = 2.0000000


array([-562.05767122, -561.82982452, -561.81265255, -561.75268257,
       -561.74797036, -561.74060798, -561.73424165, -561.6820738 ,
       -561.64206254, -561.60746824, -561.58687395])

Compute CCSD with PySCF with non-trivial active space.
Explain the `ActiveSpaceMolecularOrbitals` object in qp. How to use it to identify active orbitals

In [18]:
from quri_parts.chem.mol.models import OrbitalType

asmo = ActiveSpaceMolecularOrbitals(
    PySCFMolecularOrbitals(mole, mf.mo_coeff), active_space
)

forzen_orbs = [i for i in range(mole.nao) if asmo.orb_type(i) != OrbitalType.ACTIVE]

ccsd = cc.CCSD(mf, frozen=forzen_orbs).run(verbose=0)

In [19]:
from typing import Sequence

import numpy.typing as npt


def ccsd_param_to_circuit_param(
    uccsd: TrotterUCCSD,
    n_electrons: int,
    t1: npt.NDArray[np.complex128],
    t2: npt.NDArray[np.complex128],
) -> Sequence[float]:
    in_param_list = uccsd.param_mapping.in_params
    param_list = []

    for param in in_param_list:
        name_split = param.name.split("_")
        if name_split[0] == "s":
            _, i_str, j_str = name_split
            i, j = int(i_str), int(j_str) - n_electrons // 2
            param_list.append(t1[i, j])

        if name_split[0] == "d":
            _, i_str, j_str, a_str, b_str = name_split
            i, j, b, a = (
                int(i_str),
                int(j_str),
                int(b_str) - n_electrons // 2,
                int(a_str) - n_electrons // 2,
            )
            param_list.append(t2[i, j, a, b])
    return param_list


In [33]:
from quri_parts.qulacs.estimator import (
    create_qulacs_vector_concurrent_parametric_estimator,
)
from quri_parts.core.state import quantum_state, apply_circuit

TROTTER_STEPS = 1
USE_SINGLES = True
REDUCE_PARAMETER = True

uccsd = TrotterUCCSD(
    active_space.n_active_orb * 2,
    active_space.n_active_ele,
    trotter_number=TROTTER_STEPS,
    use_singles=USE_SINGLES,
    singlet_excitation=REDUCE_PARAMETER,
)
param = ccsd_param_to_circuit_param(uccsd, active_space.n_active_ele, ccsd.t1, ccsd.t2)
hf_state = quantum_state(active_space.n_active_orb * 2, bits=2**active_space.n_active_ele - 1)
state = apply_circuit(uccsd, hf_state)
uccsd_bound_state = state.bind_parameters(param)

In [ ]:
from scipy.special import binom

def get_cisd_size(active_space: ActiveSpace) -> int:
    n_orb = active_space.n_active_orb
    occ = active_space.n_active_ele // 2
    un_occ = n_orb - occ
    single = binom(occ, 1) * binom(un_occ, 1) * 2
    double = binom(occ, 2) * binom(un_occ, 2) * 2 + (binom(occ, 1) * binom(un_occ, 1))**2
    return int(1 + single + double)

cisd_size = get_cisd_size(active_space)

In [73]:
cisd_solver = ci.CISD(mf, forzen_orbs)
cisd_solver.nstates = 11
cisd_solver.run().e_tot

RCISD root 0  E = -562.0576692731087
RCISD root 1  E = -561.8097497841408
RCISD root 2  E = -561.7386761840094
RCISD root 3  E = -561.6820484814361
RCISD root 4  E = -561.6419212432834
RCISD root 5  E = -561.5697350286264
RCISD root 6  E = -561.4675268864721
RCISD root 7  E = -561.4306455754595
RCISD root 8  E = -561.404033751234
RCISD root 9  E = -561.3631207402732
RCISD root 10  E = -561.3208463907112


array([-562.05766927, -561.80974978, -561.73867618, -561.68204848,
       -561.64192124, -561.56973503, -561.46752689, -561.43064558,
       -561.40403375, -561.36312074, -561.32084639])

# QSCI

Show UCCSD QSCI gives good approximation to GS energy.

In [86]:
from quri_parts.qulacs.sampler import create_qulacs_general_vector_sampler
from quri_parts_qsci import qsci

BASIS_STATES = None
TOTAL_SHOTS = 10000

sampler = create_qulacs_general_vector_sampler()

bound_state = state.bind_parameters(param)
eigs, _ = qsci(
    hamiltonian, [bound_state], sampler, total_shots=TOTAL_SHOTS, num_states_pick_out=BASIS_STATES
)


Explain 
1. QSCI for excited states
2. How to build the cost function 

In [95]:
import functools
from quri_parts.core.state import ComputationalBasisSuperposition

def get_qsci_basis_size(qsci_vecs: Sequence[ComputationalBasisSuperposition]) -> int:
    return len(
        set(functools.reduce(
            lambda x, y: x + y, [[b.bits for b in s[1]] for s in qsci_vecs]
        ))
    )

In [98]:
from typing import Sequence, Callable
import numpy as np
import numpy.typing as npt
from scipy.optimize import minimize
from quri_parts.core.state import quantum_state
from quri_parts.core.sampling import ConcurrentSampler
from quri_parts.core.operator import Operator
from quri_parts.core.state import CircuitQuantumState
from quri_parts.core.utils.recording import Recorder, recordable

def create_state(bits: int, params: npt.NDArray[np.float64]) -> CircuitQuantumState:
    approx_circuit = uccsd.bind_parameters(params)
    approx_exc_state = quantum_state(approx_circuit.qubit_count, circuit=approx_circuit, bits=bits)
    return approx_exc_state


@recordable
def get_cost(
    recorder: Recorder,
    exc_bits: Sequence[int], 
    sampler: ConcurrentSampler, 
    hamiltonian: Operator, 
    fixed_state: Sequence[CircuitQuantumState]
) -> Callable[[npt.NDArray[np.float64]], float]:
    def f(x: npt.NDArray[np.float64]) -> float:
        approx_exc_state = create_state(bits=exc_bits, params=x)
        quantum_states = fixed_state + [approx_exc_state]
        vals, vecs = qsci(
            hamiltonian,
            quantum_states,
            sampler,
            total_shots=TOTAL_SHOTS,
            num_states_pick_out=BASIS_STATES
        )
        recorder.info(f"QSCI basis size ({len(fixed_state)})-th excitation", get_qsci_basis_size(vecs))
        return np.sum(vals)

    return f

Explain optimization loop. Include grad-based optimizer doesn;t make sense

In [99]:
from quri_parts.core.utils.recording import INFO, RecordSession

# Create a record session and set the record level
session = RecordSession()
session.set_level(INFO, get_cost)

sampler = create_qulacs_general_vector_sampler()

exc_masks = [0 for _ in range(10)]
fixed_state = [uccsd_bound_state]

with session.start():
    for exc_mask in exc_masks:
        bits = (2**active_space.n_active_ele - 1) ^ exc_mask
        cost = get_cost(
            exc_bits=bits,
            sampler=sampler,
            hamiltonian=hamiltonian,
            fixed_state=fixed_state
        )

        bounds = [(-10**-1, 10**-1) for _ in range(uccsd.parameter_count)]
        sol = minimize(
            cost,
            x0=np.random.uniform(-10**-2, 10**-2, uccsd.parameter_count),
            method="COBYLA",
            bounds=bounds,
            options={"maxiter": 50, "disp": True}
        )

        excited_state = quantum_state(
            active_space.n_active_orb * 2, bits=bits, circuit=uccsd.bind_parameters(sol.x)
        )
        fixed_state.append(create_state(bits, params=sol.x))


IndexError: list index out of range

In [77]:
qsci(hamiltonian, fixed_state, sampler, TOTAL_SHOTS, BASIS_STATES)[0]

array([-562.05766927, -561.82694646, -561.80974978, -561.75263823,
       -561.74600883, -561.73867618, -561.73413462, -561.68204848,
       -561.64192124, -561.60746824, -561.58687247])

In [58]:
mc.e_tot

array([-562.05767122, -561.82982452, -561.81265255, -561.75268257,
       -561.74797036, -561.74060798, -561.73424165, -561.6820738 ,
       -561.64206254, -561.60746824, -561.58687395])

Plot

In [89]:
import functools

len(set(functools.reduce(
    lambda x, y: x + y,
    [[b.bits for b in s[1]] for s in qsci(hamiltonian, fixed_state, sampler, TOTAL_SHOTS, BASIS_STATES)[1]]
)))

36